In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 3s (80.8 kB/s)
Reading package li

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-12-04 10:51:50--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.53MB/s    in 0.2s    

2022-12-04 10:51:51 (4.53 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [4]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df_tools = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Tools_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")
df_tools.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...|           Tools|          5|            0|          0|   N|                Y|Great organizer, ...|Very nice. Will o...|2015-01-31 00:08:00|
|         US|   47910848|  RF0D1LEIF6L7|B001TJGCS0|     570955425|Nite Ize Nite Daw...| 

In [5]:
# Get the number of rows in the DataFrame.
df_tools.count()

1741100

In [6]:
# Drop null values
df_tools = df_tools.dropna()
print(df_tools.count())


1740974


In [7]:
# Removed duplicate rows
df_tools = df_tools.dropDuplicates()
df_tools.count()

1740974

# Transform the Data

In [8]:
# Print schema
df_tools.printSchema()


root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [9]:
# Changing datatypes
from pyspark.sql.types import TimestampType, IntegerType

df_tools = df_tools.withColumn("customer_id", df_tools["customer_id"].cast(IntegerType()))\
       .withColumn("product_parent", df_tools["product_parent"].cast(IntegerType()))\
       .withColumn("review_date", df_tools["review_date"].cast(TimestampType()))\
       .withColumn("star_rating", df_tools["star_rating"].cast(IntegerType()))\
       .withColumn("helpful_votes", df_tools["helpful_votes"].cast(IntegerType()))\
      .withColumn("total_votes", df_tools["total_votes"].cast(IntegerType()))\

df_tools.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



## Create the "review_id_table".

In [10]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = df_tools.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show()


+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R100T355V99IID|   40190651|B0030IRKHK|     146987318|2015-01-11 00:03:00|
|R100TPQ3931RDP|    8632815|B007S8M0FA|     944178209|2014-01-04 00:11:00|
|R1010PXKUABC7W|   45427875|B000EDVU16|     417114915|2014-01-28 00:10:00|
|R1019SWPVIE9HU|   13666734|B001BNO7AE|     401304039|2015-01-08 00:07:00|
|R101ET84WIOZ5Q|   18453943|B000ST3K3W|     659433120|2014-01-01 00:09:00|
|R101FPWBD8AJ71|   36534566|B0012RWCF6|       4853427|2010-01-10 00:11:00|
|R102PBCCJ27G4G|   51132788|B001EX35U4|     712111499|2013-01-03 00:03:00|
|R103PN3YHPJMT6|   37839622|B0013E45PS|     271576246|2014-01-05 00:12:00|
|R104D8HRTE2Y9M|   48818949|B001CBVM12|      17049624|2015-01-15 00:08:00|
|R104HRVWBFR9XB|   27403242|B0002198GY|     893679678|2015-01-12 00:01:00|
|R104VJT5ZIT9FZ|   454993

## Create the "products" Table

In [47]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = df_tools.select(["product_id", "product_title"])
products_df = products_df.dropDuplicates(["product_id", "product_title"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B000JNNWQ2|IRWIN Tools VISE-...|
|B000OECQOK|Titan 17005 Stain...|
|B0045W0MD4|The American Red ...|
|B00DQQ9ET8|de.power DP-010AA...|
|B001P30BQE|DEWALT DG5204 Pro...|
|B0002TKID8|Channellock GLS-1...|
|B003O684GO|Tajima AC-500B 3/...|
|B00T96T070|UltraFire WF-68B ...|
|B001445114|Stiletto AG-102 A...|
|B00002X1ZX|Stanley Tools 28-...|
|B0006IE7OQ|Neiko 00561A Heav...|
|B00C8STBPC|ZFE 42Pcs Brass S...|
|B00004SUQN|Malco M2006 Max20...|
|B000TCZUWM|Et18 RS Mizar Ele...|
|B000189RGO|Eklind 13218 Metr...|
|B009SB5RHS|Huntsman® - W20 4...|
|B0002IXQHO|Robert Sorby 67HS...|
|B004Q05TTC|Superior Tool 375...|
|B00012Y38W|Bondhus 20199 Bal...|
|B003YL4954|PORTER-CABLE NS18...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [12]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = df_tools.groupby("customer_id").agg({
    "customer_id": "count"
    }).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   44663040|             1|
|     979054|             1|
|   25065235|            16|
|    2584297|             1|
|   12072867|             3|
|   51812966|             3|
|    1636908|             2|
|   11883477|             2|
|   45638392|             1|
|   12049572|             8|
|   36063094|            28|
|   27475967|             1|
|    1353835|             1|
|   49204456|            10|
|   10609442|             1|
|   32921622|             1|
|   43792381|             9|
|   46588922|             7|
|   45812506|            10|
|   34513899|             3|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [13]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = df_tools.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100T355V99IID|          5|            0|          0|   N|
|R100TPQ3931RDP|          1|            0|          0|   N|
|R1010PXKUABC7W|          5|            0|          1|   N|
|R1019SWPVIE9HU|          5|            0|          0|   N|
|R101ET84WIOZ5Q|          4|            0|          0|   N|
|R101FPWBD8AJ71|          4|            1|          2|   N|
|R102PBCCJ27G4G|          1|            1|          2|   N|
|R103PN3YHPJMT6|          4|            0|          1|   N|
|R104D8HRTE2Y9M|          5|            0|          0|   N|
|R104HRVWBFR9XB|          5|            0|          0|   N|
|R104VJT5ZIT9FZ|          5|            0|          0|   N|
|R1050IM0HAVKHI|          5|            3|          4|   N|
|R105JVVCLH4MBG|          4|            0|          0|   N|
|R105O2T37XKXQA|          4|            

# Load

In [14]:
mode = "append"
jdbc_url="jdbc:postgresql://bigdatachallengedb.cy4by0s0blfh.ap-southeast-2.rds.amazonaws.com:5432/bigDataChallengeDB"
# config = {"user":"postgres", "password": "<password>", "driver":"org.postgresql.Driver"}
config = {"user":"root",
          "password": "<********>",
          "driver":"org.postgresql.Driver"}

In [15]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [48]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [17]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [18]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)